In [15]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["GROQ_API"] = os.getenv("GROQ_API")

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=os.getenv("NEO4J_URI"),username=os.getenv("NEO4J_USERNAME"),password=os.getenv("NEO4J_PASSWORD"))

ValueError: Cannot resolve address a811bc95.databases.neo4j.io:7687

In [16]:
NEO4J_URI="neo4j+s://045a8ba1.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="_AoMU1fB8R-60HpuKLTRZOz1WhuYP6uU1R_RQoWXTK0"

In [30]:
from neo4j import GraphDatabase
import csv

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def import_csv(tx, row):
    tx.run("""
    MERGE (t:Track {name: $track_name, isrc: $isrc})
    ON CREATE SET 
        t.release_date = $release_date,
        t.explicit = $explicit_track,
        t.spotify_streams = $spotify_streams,
        t.spotify_playlist_count = $spotify_playlist_count,
        t.spotify_playlist_reach = $spotify_playlist_reach,
        t.spotify_popularity = $spotify_popularity,
        t.youtube_views = $youtube_views,
        t.youtube_likes = $youtube_likes,
        t.tiktok_posts = $tiktok_posts,
        t.tiktok_likes = $tiktok_likes,
        t.tiktok_views = $tiktok_views,
        t.youtube_playlist_reach = $youtube_playlist_reach,
        t.apple_music_playlist_count = $apple_music_playlist_count,
        t.airplay_spins = $airplay_spins,
        t.siriusxm_spins = $siriusxm_spins,
        t.deezer_playlist_count = $deezer_playlist_count,
        t.deezer_playlist_reach = $deezer_playlist_reach,
        t.amazon_playlist_count = $amazon_playlist_count,
        t.pandora_streams = $pandora_streams,
        t.pandora_track_stations = $pandora_track_stations,
        t.soundcloud_streams = $soundcloud_streams,
        t.shazam_counts = $shazam_counts,
        t.tidal_popularity = $tidal_popularity
    MERGE (p1:Platform {name: "Spotify"})
    MERGE (t)-[:PLAYLISTED_ON {playlist_count: $spotify_playlist_count, playlist_reach: $spotify_playlist_reach}]->(p1)
    MERGE (t)-[:POPULAR_IN {popularity_score: $spotify_popularity}]->(p1)

    MERGE (p2:Platform {name: "YouTube"})
    MERGE (t)-[:STREAMED_VIA {stream_count: $youtube_views}]->(p2)
    MERGE (t)-[:ENGAGED_ON {post_count: $tiktok_posts, like_count: $tiktok_likes, view_count: $tiktok_views}]->(:SocialMediaPlatform {name: "TikTok"})
    """, 
    track_name=row['Track'],
    isrc=row['ISRC'],
    release_date=row['Release Date'],
    explicit_track=row['Explicit Track'],
    spotify_streams=row['Spotify Streams'],
    spotify_playlist_count=row['Spotify Playlist Count'],
    spotify_playlist_reach=row['Spotify Playlist Reach'],
    spotify_popularity=row['Spotify Popularity'],
    youtube_views=row['YouTube Views'],
    youtube_likes=row['YouTube Likes'],
    tiktok_posts=row['TikTok Posts'],
    tiktok_likes=row['TikTok Likes'],
    tiktok_views=row['TikTok Views'],
    youtube_playlist_reach=row['YouTube Playlist Reach'],
    apple_music_playlist_count=row['Apple Music Playlist Count'],
    airplay_spins=row['AirPlay Spins'],
    siriusxm_spins=row['SiriusXM Spins'],
    deezer_playlist_count=row['Deezer Playlist Count'],
    deezer_playlist_reach=row['Deezer Playlist Reach'],
    amazon_playlist_count=row['Amazon Playlist Count'],
    pandora_streams=row['Pandora Streams'],
    pandora_track_stations=row['Pandora Track Stations'],
    soundcloud_streams=row['Soundcloud Streams'],
    shazam_counts=row['Shazam Counts'],
    tidal_popularity=row['TIDAL Popularity'],
    album_name=row['Album Name'],
    artist_name=row['Artist']
    )


with driver.session() as session:
    # Change encoding to ISO-8859-1
    with open(r'C:\Users\Raghu\Downloads\archive (10)\Most Streamed Spotify Songs 2024.csv', newline='', encoding='ISO-8859-1') as f:
        reader = csv.DictReader(f)
        for row in reader:
            session.execute_write(import_csv, row)

driver.close()

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="gemma2-9b-it",api_key=os.getenv("GROQ_API"))
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020772FB3D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020772FB2410>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
from langchain.graphs import Neo4jGraph

neo4j_graph = Neo4jGraph(
    url="neo4j+s://045a8ba1.databases.neo4j.io:7687",
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Optional: Check if the connection was successful by printing the schema
print(neo4j_graph.schema)  # This should output the schema if the connection is successfu

Node properties:
Track {name: STRING, isrc: STRING, release_date: STRING, explicit: STRING, spotify_streams: STRING, spotify_playlist_count: STRING, spotify_playlist_reach: STRING, spotify_popularity: STRING, youtube_views: STRING, youtube_likes: STRING, tiktok_posts: STRING, tiktok_likes: STRING, tiktok_views: STRING, youtube_playlist_reach: STRING, apple_music_playlist_count: STRING, airplay_spins: STRING, siriusxm_spins: STRING, deezer_playlist_count: STRING, deezer_playlist_reach: STRING, amazon_playlist_count: STRING, pandora_streams: STRING, pandora_track_stations: STRING, soundcloud_streams: STRING, shazam_counts: STRING, tidal_popularity: STRING}
Platform {name: STRING}
SocialMediaPlatform {name: STRING}
Relationship properties:
PLAYLISTED_ON {playlist_count: STRING, playlist_reach: STRING}
POPULAR_IN {popularity_score: STRING}
STREAMED_VIA {stream_count: STRING}
ENGAGED_ON {like_count: STRING, post_count: STRING, view_count: STRING}
The relationships:
(:Track)-[:PLAYLISTED_ON]

In [19]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=neo4j_graph,llm=llm,verbose = True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000002076BACB790>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020772FB3D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020772FB2410>, model_name=

In [20]:
response = chain.invoke({"query": "How many tracks are available on spotify that have a spotify popularity score > 80"})
print(response['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Track)<-[:POPULAR_IN]-(p:Platform {name:"Spotify"}) WHERE t.spotify_popularity > 80 RETURN count(t)
Full Context:
[{'count(t)': 0}]

> Finished chain.
I don't know the answer. 



# Few-shot Prompting

In [22]:
examples = [
    {
        "question": "How many tracks are available on Spotify that have a Spotify popularity score > 80?",
        "query": 'MATCH (t:Track)<-[:POPULAR_IN]-(p:Platform {name: "Spotify"}) WHERE t.spotify_popularity > 80 RETURN count(t)'
    },
    {
        "question": "List the top 5 most popular tracks on YouTube by view count.",
        "query": 'MATCH (t:Track)-[:STREAMED_VIA]->(p:Platform {name: "YouTube"}) RETURN t.name, t.youtube_views ORDER BY t.youtube_views DESC LIMIT 5'
    },
    {
        "question": "Which tracks have more than 1 million streams on SoundCloud?",
        "query": 'MATCH (t:Track) WHERE t.soundcloud_streams > 1000000 RETURN t.name, t.soundcloud_streams'
    },
    {
        "question": "Find tracks with a Shazam count greater than 500,000.",
        "query": 'MATCH (t:Track) WHERE t.shazam_counts > 500000 RETURN t.name, t.shazam_counts'
    },
    {
        "question": "What are the top 3 TikTok tracks by number of posts?",
        "query": 'MATCH (t:Track)-[:ENGAGED_ON]->(:SocialMediaPlatform {name: "TikTok"}) RETURN t.name, t.tiktok_posts ORDER BY t.tiktok_posts DESC LIMIT 3'
    },
    {
        "question": "How many explicit tracks are available across all platforms?",
        "query": 'MATCH (t:Track) WHERE t.explicit = true RETURN count(t)'
    },
    {
        "question": "What is the average Spotify popularity score for tracks released after 2020?",
        "query": 'MATCH (t:Track) WHERE t.release_date > date("2020-01-01") RETURN avg(t.spotify_popularity) AS avg_popularity'
    },
    {
        "question": "List all tracks available on both Spotify and YouTube.",
        "query": 'MATCH (t:Track)-[:PLAYLISTED_ON]->(p1:Platform {name: "Spotify"}), (t)-[:STREAMED_VIA]->(p2:Platform {name: "YouTube"}) RETURN t.name'
    },
    {
        "question": "What are the top 5 most-played tracks on Amazon Music by playlist count?",
        "query": 'MATCH (t:Track)-[:PLAYLISTED_ON]->(p:Platform {name: "Amazon Music"}) RETURN t.name, t.amazon_playlist_count ORDER BY t.amazon_playlist_count DESC LIMIT 5'
    },
    {
        "question": "How many tracks have more than 1 million views on YouTube and a popularity score over 80 on Spotify?",
        "query": 'MATCH (t:Track)-[:STREAMED_VIA]->(p1:Platform {name: "YouTube"}), (t)<-[:POPULAR_IN]-(p2:Platform {name: "Spotify"}) WHERE t.youtube_views > 1000000 AND t.spotify_popularity > 80 RETURN count(t)'
    }
]

In [27]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Define example prompt template
example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)

# Define the few-shot prompt template with examples and input format
prompt = FewShotPromptTemplate(
    examples=examples[:5],  # Use the first 5 examples
    example_prompt=example_prompt,
    prefix="You are a neo4j expert. Given an input question, create a very accurate Cypher query.",
    suffix="User input: {question}\nCypher query:",
    input_variables=["question"]  # Ensure this matches the variables in prefix/suffix
)

# Test with an input question
question = "Find the top 10 tracks by Spotify popularity."
formatted_prompt = prompt.format(question=question)

print(formatted_prompt)


KeyError: 'name'

In [26]:
question = "Find the top 10 tracks by Spotify popularity."
formatted_prompt = prompt.format(question=question)

print(formatted_prompt)

KeyError: 'name'